In [1]:
%load_ext autoreload
%autoreload 2
import datetime, urllib, re, math, json, requests,io
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
from scipy import optimize
import scipy.signal
from sklearn.linear_model import LinearRegression
from IPython.display import display, HTML
from ipywidgets import *


try:
    from packaging import version
except ImportError:
    !pip install packaging

if( version.parse(pd.__version__) < version.parse("0.23.4")):
    print("update pandas")
    !pip install pandas --upgrade #--ignore-installed
    print(version.parse(pd.__version__))

    
plt.rcParams['figure.dpi'] = 200
    
import compute    

In [5]:
#get the data
url = "https://coronadatascraper.com/timeseries-tidy.csv.zip"
df =pd.read_csv(url, compression='zip')
dataTCS =df.set_index(["city","county","state","country","type","level"])
dataTCS["date"] = [datetime.datetime.strptime(ts,"%Y-%m-%d").date() for ts in dataTCS["date"]]
if (datetime.date.today() - dataTCS["date"].unique()[-1]).days >2: assert(0)
print("latest sample is at %s" %dataTCS["date"].unique()[-1])

latest sample is at 2020-04-07


In [6]:
compute.dataTCS = dataTCS

In [139]:
import compute
compute.dataTCS = dataTCS
from ipywidgets import *



def updateEqualizeTrg_e(*args):
    equalizeTrg_w.options = compute.getList(level_w.value,country_w.value)
    equalizeTrg_w.value=equalizeTrg_w.options[0]
 

def country_e(*args):
    if(level_w.value=="country"):
        country_w.options=[""]
        country_w.value  = ""
    elif(level_w.value=="state"):
        #must do option then value!
        country_w.options = compute.getList("country")                
        country_w.value   = "United States"
        
 
equalizeTrg_w = widgets.Dropdown(
    options    = compute.getList("country"),
    value      = "United States",
    description='equalizeTrg',
    disabled   = False,
)

# parent for equalization, can be country or state
# no parent if level is county
country_w = widgets.Dropdown(
    options    = compute.getList("country"),
    value      = "United States",
    description='Country',
    disabled   = False,
)

level_w = widgets.Dropdown(
    options    = ["country","state"],
    value      = 'country',
    description='level',
    disabled   = False,
)
#when level changes update parent
level_w.observe(country_e, 'value')
#when level changes update equalize target (child of parent ie, country--> state)
level_w.observe(updateEqualizeTrg_e)
country_w.observe(updateEqualizeTrg_e)


interact(compute.plotter,
         minCount     = (0,1e6,1e3),
         minCountKpi  = dataTCS.index.levels[4].unique().values,
         equalizeTrg  = equalizeTrg_w,
         level        = level_w,
         equalizeCount= (0,10000),
         country       = country_w,
         plotKpi=dataTCS.index.levels[4].unique().values
         ,xlim        = (-1,60)
         ,ylim        = (-1,1e6)
#         equalize=[True, False],         
#         logyPlot=[True, False],        
#         reference=[True,False]
        );#countryStateSweep=dataTCS.index.levels[3].unique().values)

interactive(children=(FloatSlider(value=5000.0, description='minCount', max=1000000.0, step=1000.0), Dropdown(…

In [ ]:
import compute
compute.dataTCS = dataTCS
country = "United States"
level   = "state"
numRows = 12
interact(
    compute.table_gen,
    country="United States",
    level = ["country","state"], 
    numRows=(3,50,1));
# country -> user period to combine states acorss countries

In [7]:
import compute
compute.dataTCS = dataTCS
from ipywidgets import *

#country
level_w = widgets.Dropdown(
    options=["country","state","county"],
    value='state',
    description='level',
    disabled=False,
)
target_w = widgets.Dropdown(
    options    = compute.getList(level_w.value),
    value      = list(compute.getList(level_w.value).items())[0][1],
    description= 'State',
    disabled   = False,
)

def target_e(*args):
    target_w.options = compute.getList(level_w.value)


level_w.observe(target_e, 'value')



interact(compute.projector,
    stype   = dataTCS.index.levels[4],
    level   = level_w,
    target  = target_w,    
    minCount= (0,1e5,1e2),
    dteOffset=(-20,0),
    smoothLength=(3,13,2), 
    smoothOrder=(1,5)
        );

interactive(children=(Dropdown(description='stype', index=1, options=('active', 'cases', 'deaths', 'growthFact…